In [1]:
import pymrio as mr
import pandas as pd
import numpy as np
from scipy.linalg import block_diag
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
import os
%matplotlib inline

In [2]:
# Pour enregistrer/charger rapidement
import pickle

In [3]:
EXIOBASE_PATH = 'EXIOBASE/' # ne pas hésiter à changer

In [4]:
# Charger
with open(EXIOBASE_PATH + f'IOT_2019_ixi.pickle', 'rb') as handle:
    io2019 = pickle.load(handle)

In [5]:
io2019.impacts.get_index()

Index(['Value Added', 'Employment', 'Employment hour',
       'GHG emissions (GWP100) | Problem oriented approach: baseline (CML, 2001) | GWP100 (IPCC, 2007)',
       'Human toxicity (USEtox) | USEtox2008 | CTUh (Rosenbaum et al., 2008)',
       'Fresh water Ecotoxicity (USEtox) | USEtox2008 | CTUe (Rosenbaum et al., 2008)',
       'EPS | Damage Approach | EPS (Steen, 1999))',
       'Carcinogenic effects on humans (H.A) | ECOINDICATOR 99 (H.A) | Carcinogenic effects on humans (H.A)',
       'Respiratory effects on humans caused by organic substances (H.A) | ECOINDICATOR 99 (H.A) | Respiratory effects on humans caused by organic substances (H.A)',
       'Respiratory effects on humans caused by inorganic substances (H.A) | ECOINDICATOR 99 (H.A) | Respiratory effects on humans caused by inorganic substances (H.A)',
       ...
       'Unused Domestic Extraction - Crop and Crop Residue',
       'Unused Domestic Extraction - Grazing and Fodder',
       'Unused Domestic Extraction - Forestr

# Agrégation

In [6]:
new_regions = ['EUR'] * 28 + ['NAM'] + ['APAC'] + ['CN'] + ['NAM'] + ['APAC'] + ['LAM'] + ['IN'] + ['NAM'] + ['RU'] + ['APAC'] + ['EUR'] + ['EUR'] + ['APAC'] + ['EUR'] + ['APAC'] + ['AFR'] + ['APAC'] + ['LAM'] + ['EUR'] + ['AFR'] + ['MEA']

io2019_reg_agg = io2019.aggregate(region_agg=new_regions, inplace=False)

In [7]:
io2019_reg_agg.calc_all()

In [8]:
# Order of results dataframe

# COLUMNS
#
# Per M€
# - Luxembourg
#   - direct (S)
#   - lifecycle (M)
# - EU28
#   - direct (S)
#   - lifecycle (M)
# Total
# - Luxembourg
#   - direct (D_pba)
#   - lifecycle (D_cba)
# - EU28
#   - direct (D_pba)
#   - lifecycle (D_cba)
#
# ROWS
# Indicators

# Per M€
factors = [io2019.impacts.S['LU'],
           io2019.impacts.M['LU'],
           io2019_reg_agg.impacts.S['EUR'],
           io2019_reg_agg.impacts.M['EUR']]

# Total
impacts = [io2019.impacts.D_pba['LU'],
           io2019.impacts.D_cba['LU'],
           io2019_reg_agg.impacts.D_pba['EUR'],
           io2019_reg_agg.impacts.D_cba['EUR']]
           

In [9]:
results = pd.concat([pd.concat(factors, axis=1, keys=['LU, direct',
                                                      'LU, indirect',
                                                      'EU28, direct',
                                                      'EU28, indirect']),
                     pd.concat(impacts, axis=1, keys=['LU, direct',
                                                      'LU, indirect',
                                                      'EU28, direct',
                                                      'EU28, indirect'])], axis=1, keys=['Per M€', 'Total'])

# Secteurs MECO

In [10]:
C_sectors = pd.read_excel('MECO_sectors.xlsx', sheet_name='EXIO_sectors', index_col=[0,1,2,3])

In [11]:
io2019_sec_agg = io2019.aggregate(sector_agg=C_sectors.values.T,
                                  sector_names=C_sectors.columns.tolist(),
                                  inplace=False)

In [12]:
io2019_sec_agg.calc_all()

In [13]:
io2019_reg_sec_agg = io2019.aggregate(sector_agg=C_sectors.values.T,
                                      sector_names=C_sectors.columns.tolist(),
                                      region_agg=new_regions,
                                      inplace=False)

In [14]:
io2019_reg_sec_agg.calc_all()

In [15]:
# Per M€
factors = [io2019_sec_agg.impacts.S['LU'],
           io2019_sec_agg.impacts.M['LU'],
           io2019_reg_sec_agg.impacts.S['EUR'],
           io2019_reg_sec_agg.impacts.M['EUR']]

# Total
impacts = [io2019_sec_agg.impacts.D_pba['LU'],
           io2019_sec_agg.impacts.D_cba['LU'],
           io2019_reg_sec_agg.impacts.D_pba['EUR'],
           io2019_reg_sec_agg.impacts.D_cba['EUR']]
           

In [16]:
results = pd.concat([pd.concat(factors, axis=1, keys=['LU, direct',
                                                      'LU, indirect',
                                                      'EU28, direct',
                                                      'EU28, indirect']),
                     pd.concat(impacts, axis=1, keys=['LU, direct',
                                                      'LU, indirect',
                                                      'EU28, direct',
                                                      'EU28, indirect'])], axis=1, keys=['Per M€', 'Total'])

In [17]:
results.columns.names = ['unit', 'factor', 'sector']

In [18]:
ind = 'GHG emissions (GWP100) | Problem oriented approach: baseline (CML, 2001) | GWP100 (IPCC, 2007)'

In [19]:
results.loc[ind].unstack()

sector                 C - Manufacturing  C10 - Manufacture of food products  \
unit   factor                                                                  
Per M€ LU, direct                    0.0                        3.486801e+04   
       LU, indirect                  0.0                        5.924535e+04   
       EU28, direct                  0.0                        4.679056e+04   
       EU28, indirect                0.0                        7.582584e+04   
Total  LU, direct                    0.0                        2.553881e+07   
       LU, indirect                  0.0                        1.602965e+09   
       EU28, direct                  0.0                        5.432943e+10   
       EU28, indirect                0.0                        5.339154e+11   

sector                 C11 - Manufacture of beverages  \
unit   factor                                           
Per M€ LU, direct                        5.985801e+04   
       LU, indirect                      8.965147e+04   
       EU28, direct                      6.969760e+04   
       EU28, indirect                    1.106627e+05   
Total  LU, direct                        4.789217e+06   
       LU, indirect                      1.185031e+08   
       EU28, direct                      1.373158e+10   
       EU28, indirect                    5.690739e+10   

sector                 C12 - Manufacture of tobacco products  \
unit   factor                                                  
Per M€ LU, direct                               8.059103e+03   
       LU, indirect                             5.340328e+04   
       EU28, direct                             1.346363e+05   
       EU28, indirect                           1.562926e+05   
Total  LU, direct                               2.774185e+06   
       LU, indirect                             7.698085e+07   
       EU28, direct                             4.953988e+09   
       EU28, indirect                           1.234022e+10   

sector                 C13 - Manufacture of textiles  \
unit   factor                                          
Per M€ LU, direct                       2.718441e+04   
       LU, indirect                     1.707891e+05   
       EU28, direct                     4.414312e+04   
       EU28, indirect                   1.459893e+05   
Total  LU, direct                       4.599951e+07   
       LU, indirect                     4.790095e+08   
       EU28, direct                     9.162795e+09   
       EU28, indirect                   6.512226e+10   

sector                 C14 - Manufacture of wearing apparel  \
unit   factor                                                 
Per M€ LU, direct                              1.012899e+07   
       LU, indirect                            1.014300e+07   
       EU28, direct                            5.294902e+04   
       EU28, indirect                          1.225511e+05   
Total  LU, direct                              4.593821e+07   
       LU, indirect                            2.237310e+08   
       EU28, direct                            8.941796e+09   
       EU28, indirect                          8.222300e+10   

sector                 C15 - Manufacture of leather and related products  \
unit   factor                                                              
Per M€ LU, direct                                           0.000000e+00   
       LU, indirect                                         0.000000e+00   
       EU28, direct                                         1.687358e+04   
       EU28, indirect                                       6.962488e+04   
Total  LU, direct                                           0.000000e+00   
       LU, indirect                                         1.100473e+08   
       EU28, direct                                         1.246254e+09   
       EU28, indirect                                       3.881366e+10   

sector                 C16 - Manuf

In [20]:
results.loc[ind].unstack().T.to_excel('MECO_sector_GHGs.xlsx')

In [21]:
results.to_excel('MECO_sector_all_factors.xlsx')

In [22]:
io2019.impacts.unit.to_clipboard()